In [ ]:
# CaisisImport. A utility to prepare a Caisis-to-Excel export for import into Oncoscape (via cBioPortal format).

# Steps:
# . ExportAsTSVs
# . ZeroDates
# . LimitByDiseaseGroups
# . 


In [67]:
import os
import errno
def save_file_and_folder(filename, obj):
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    with open(filename, "w") as f:
        f.write(obj)

def patients_and_descriptive_header(df:pd.DataFrame, header):
    ids = df['PatientId'].values
    str_ids = "\r\n".join(ids)
    a =  header + "\r\n" + str_ids
    return a

In [79]:
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd

dataset_folder = '../Caisis_NonPublicData/Brain'
diseaseChosen = 'Brain'

knownTables = ['Demographics', 'Status', 'Encounters', 'ClinicalStage', 'LabTests', 'PathTest', 'SocialHistory', 'LabTestGenetics', 'PathStageGrade', 'RadiationTherapy',  ]
#print(knownTables)

table_demographics = pd.read_csv(dataset_folder +'/RawCSVs/' + knownTables[1] +'.csv') #'Demographics.csv')
table_demographics['Date'] =pd.to_datetime(table_demographics.StatusDate)
table_demographics['PatientId'] =  table_demographics['PatientId'].astype(str)

dxRows = table_demographics[table_demographics.Status.eq('Diagnosis Date')]
diseaseRows = dxRows[dxRows.StatusDisease.eq(diseaseChosen)].sort_values('Date')
#print(diseaseRows)

dxNanRows = diseaseRows[diseaseRows.Date.isna()]
if(dxNanRows.size > 0):
    print('CHECK NoDxDate? REPORT NoDxDate.txt has *' +str(dxNanRows.size)+'* diagnoses of '+diseaseChosen+' without diagnosis dates.')
    report_body = patients_and_descriptive_header(dxNanRows, 'Patients with "'+diseaseChosen+" but no DiagnosisDate:")
    save_file_and_folder('reports/NoDxDate.txt', report_body)
else:
    print('CHECK NoDxDate? OK')
    print('TBD: delete existing NoDxDate.txt report.')

diseaseDatedRows = diseaseRows[diseaseRows.Date.isna()==False]
#print(diseaseRows[diseaseRows.PatientId.eq('1920')])
print('----')
patientid_date_dict = {}
for index, row in diseaseDatedRows.iterrows():
    pid = str(row['PatientId'])
    if((pid in patientid_date_dict) == False):
        patientid_date_dict[pid] = row['Date']
    else:
        pass
print('Resulting patient IDs = ' + str(len(patientid_date_dict)))    
#print(patientid_date_dict['1920'])


CHECK NoDxDate? REPORT NoDxDate.txt has *30* diagnoses of Brain without diagnosis dates.
----
Resulting patient IDs = 1775
